# Part 1
This part is concerned with loading images and displaying them.

## Problem 1
### Loading the images
First we want to load the images into a numpy array. Before we do that, let's set up some imports and paths.


In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import convolve, convolve1d
from PIL import Image

folder_path = "toyProblem_F22"

### Load into a NumPy array
We then want to store the data in a NumPy array.
Along the way we are tasked to convert to grayscale and normalise to the range [0, 1].

In [ ]:
image_files = sorted([f for f in os.listdir(folder_path)])

num_images = len(image_files)
image_shape = (256, 256)
image_sequence = np.zeros((num_images, *image_shape), dtype=np.float32)

for i, file in enumerate(image_files):
    img_path = os.path.join(folder_path, file)
    img = Image.open(img_path).convert("L") # this converts to grayscale
    img_array = np.array(img, dtype=np.float32) / 255.0 # this normalises to [0, 1]
    image_sequence[i] = img_array

### Showing the images
We then want to show the images. We do this by showing each image and adding a delay of 0.1 seconds, clearing the frame and restarting.

In [ ]:
plt.figure()
for frame in range(num_images):
    plt.imshow(image_sequence[frame], cmap="gray")
    plt.title(f"Frame {frame+1}/{num_images}")
    plt.axis("off")
    plt.pause(0.1)
    plt.clf()

plt.close()

# Part 2
This part is concerned with image gradients.

## Problem 2.1
### Low level gradient calculation
We can use the simple low level gradient calculation to calculate the gradients of the images.

In [ ]:
V_x = image_sequence[:, :, 1:] - image_sequence[:, :, :-1]
V_y = image_sequence[:, 1:, :] - image_sequence[:, :-1, :]
V_t = image_sequence[1:, :, :] - image_sequence[:-1, :, :]

### Showing the gradients of a selected frame
We can show the gradients of a selected frame by showing the gradients of the frame.

In [ ]:
selected_frame = 10 # we select the 11th frame for no particular reason
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(V_x[selected_frame], cmap="gray")
plt.title("V_x (Gradient in x-direction)")
plt.axis("off")

plt.subplot(1, 3, 2)
plt.imshow(V_y[selected_frame], cmap="gray")
plt.title("V_y (Gradient in y-direction)")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.imshow(V_t[selected_frame], cmap="gray")
plt.title("V_t (Gradient in time)")
plt.axis("off")

plt.tight_layout()
plt.show()

### Observations of the gradients
We observe that the gradients are split between the x, y and t directions
- The x gradient shows the changes in the vertical direction
- The y gradient shows the changes in the horizontal direction
- The t gradient shows the regions where motion occurs between frames

## Problem 2.2
### Simple gradient filters
We want to use prewitt and sobel kernels to calculate the gradients. Let's define these kernels - they can be found [here](https://www.projectrhea.org/rhea/index.php/An_Implementation_of_Sobel_Edge_Detection).

In [ ]:
prewitt_x = np.array([[-1, 0, 1],
                      [-1, 0, 1],
                      [-1, 0, 1]])

prewitt_y = np.array([[-1, -1, -1],
                      [0, 0, 0],
                      [1, 1, 1]])

sobel_x = np.array([[-1, 0, 1],
                    [-2, 0, 2],
                    [-1, 0, 1]])

sobel_y = np.array([[-1, -2, -1],
                    [0, 0, 0],
                    [1, 2, 1]])

#### Convolution with the kernels
We can then convolve the kernels with the images to get the gradients.

In [ ]:
V_x_prewitt = convolve(image_sequence, prewitt_x[np.newaxis, :, :], mode="constant")
V_y_prewitt = convolve(image_sequence, prewitt_y[np.newaxis, :, :], mode="constant")

V_x_sobel = convolve(image_sequence, sobel_x[np.newaxis, :, :], mode="constant")
V_y_sobel = convolve(image_sequence, sobel_y[np.newaxis, :, :], mode="constant")

### Showing the gradients of a selected frame
We can show the gradients of a selected frame by showing the gradients of the frame.
We use the scipy.ndimage.convolve function to convolve the image with the kernel. Documentation can be found [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.convolve.html).

In [ ]:
plt.figure(figsize=(15, 10))

plt.subplot(2, 2, 1)
plt.imshow(V_x_prewitt[selected_frame], cmap="gray")
plt.title("V_x (Prewitt)")

plt.subplot(2, 2, 2)
plt.imshow(V_y_prewitt[selected_frame], cmap="gray")
plt.title("V_y (Prewitt)")

plt.subplot(2, 2, 3)
plt.imshow(V_x_sobel[selected_frame], cmap="gray")
plt.title("V_x (Sobel)")

plt.subplot(2, 2, 4)
plt.imshow(V_y_sobel[selected_frame], cmap="gray")
plt.title("V_y (Sobel)")

plt.tight_layout()
plt.show()

I think we should not keep these, since scipy convolve already handles flipping for us.

### Flipping the kernels and plotting again

### Image Filtering

In [ ]:
temporal_filter = np.array([[-1], [1]])

V_x_prewitt = convolve(image_sequence, prewitt_x[np.newaxis, :, :], mode="constant")
V_y_prewitt = convolve(image_sequence, prewitt_y[np.newaxis, :, :], mode="constant")

V_x_sobel = convolve(image_sequence, sobel_x[np.newaxis, :, :], mode="constant")
V_y_sobel = convolve(image_sequence, sobel_y[np.newaxis, :, :], mode="constant")

V_t = convolve(image_sequence, temporal_filter.reshape(2, 1, 1), mode="constant")


### Plot this

In [ ]:
selected_frame = 10

fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 10))

# Prewitt gradients (first row)
axes[0, 0].imshow(V_x_prewitt[selected_frame], cmap="gray")
axes[0, 0].set_title("V_x (Prewitt)")
axes[0, 0].axis("off")

axes[0, 1].imshow(V_y_prewitt[selected_frame], cmap="gray")
axes[0, 1].set_title("V_y (Prewitt)")
axes[0, 1].axis("off")

axes[0, 2].imshow(V_t[selected_frame], cmap="gray")
axes[0, 2].set_title("V_t (Prewitt)")
axes[0, 2].axis("off")

# Sobel gradients (second row)
axes[1, 0].imshow(V_x_sobel[selected_frame], cmap="gray")
axes[1, 0].set_title("V_x (Sobel)")
axes[1, 0].axis("off")

axes[1, 1].imshow(V_y_sobel[selected_frame], cmap="gray")
axes[1, 1].set_title("V_y (Sobel)")
axes[1, 1].axis("off")

axes[1, 2].imshow(V_t[selected_frame], cmap="gray")
axes[1, 2].set_title("V_t (Sobel)")
axes[1, 2].axis("off")

plt.tight_layout()
plt.show()

## Problem 2.3
### Gaussian Gradient Filters

In [ ]:
def Gauss(x,sigma):
    return (1/(np.sqrt(2*np.pi*(sigma**2))))*np.exp(-(x**2)/(2*(sigma**2)))

# an array of a given size with a gaussian distribution in dicrete steps

def GaussArray(sigma,size):
    arr = np.arange(-sigma*size,sigma*size+1,1)
    G = Gauss(arr,sigma)
    return G / np.sum(np.abs(G))

# the derrivative of the gaussian function

def dGauss(x,sigma):
    return (x/(sigma**2))*Gauss(x,sigma)

# an array of a given size with the derrivative of a gaussian distribution in discrete steps

def dGaussArray(sigma,size):
    arr = np.arange(-sigma*size,sigma*size+1,1)
    G = dGauss(arr,sigma)
    return G / np.linalg.norm(G)

def V_x_gauss(sigma,size):
    V = convolve1d(image_sequence,dGaussArray(sigma,size),2)
    return V

def V_y_gauss(sigma,size):
    V = convolve1d(image_sequence,dGaussArray(sigma,size),1)
    return V

def V_t_gauss(sigma,size):
    V = convolve1d(image_sequence,dGaussArray(sigma,size),0)
    return V

Plot

In [ ]:
plt.figure(figsize=(15, 10))

plt.subplot(2, 3, 1)
plt.imshow(V_x_gauss(1,2)[selected_frame], cmap="gray")
plt.title("V_x (Gauss)")

plt.subplot(2, 3, 2)
plt.imshow(V_y_gauss(1,2)[selected_frame], cmap="gray")
plt.title("V_y (Gauss)")

plt.subplot(2, 3, 3)
plt.imshow(V_t[selected_frame], cmap="gray")
plt.title("V_t (Gauss)")

plt.subplot(2, 3, 4)
plt.imshow(V_x_sobel[selected_frame], cmap="gray")
plt.title("V_x (Sobel)")

plt.subplot(2, 3, 5)
plt.imshow(V_y_sobel[selected_frame], cmap="gray")
plt.title("V_y (Sobel)")

plt.subplot(2, 3, 6)
plt.imshow(V_t[selected_frame], cmap="gray")
plt.title("V_t (Sobel)")

plt.tight_layout()
plt.show()

In [ ]:
selected_frame = 10

fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(15, 15))

# Row 1: Gaussian gradients
axes[0, 0].imshow(V_x_gauss(1, 2)[selected_frame], cmap="gray")
axes[0, 0].set_title("V_x (Gauss)")
axes[0, 0].axis("off")

axes[0, 1].imshow(V_y_gauss(1, 2)[selected_frame], cmap="gray")
axes[0, 1].set_title("V_y (Gauss)")
axes[0, 1].axis("off")

axes[0, 2].imshow(V_t[selected_frame], cmap="gray")
axes[0, 2].set_title("V_t (Gauss)")
axes[0, 2].axis("off")

# Row 2: Sobel gradients
axes[1, 0].imshow(V_x_sobel[selected_frame], cmap="gray")
axes[1, 0].set_title("V_x (Sobel)")
axes[1, 0].axis("off")

axes[1, 1].imshow(V_y_sobel[selected_frame], cmap="gray")
axes[1, 1].set_title("V_y (Sobel)")
axes[1, 1].axis("off")

axes[1, 2].imshow(V_t[selected_frame], cmap="gray")
axes[1, 2].set_title("V_t (Sobel)")
axes[1, 2].axis("off")

# Row 3: Prewitt gradients
axes[2, 0].imshow(V_x_prewitt[selected_frame], cmap="gray")
axes[2, 0].set_title("V_x (Prewitt)")
axes[2, 0].axis("off")

axes[2, 1].imshow(V_y_prewitt[selected_frame], cmap="gray")
axes[2, 1].set_title("V_y (Prewitt)")
axes[2, 1].axis("off")

axes[2, 2].imshow(V_t[selected_frame], cmap="gray")
axes[2, 2].set_title("V_t (Prewitt)")
axes[2, 2].axis("off")

plt.tight_layout()
plt.show()


## Problem 3.1

In [ ]:
selected_frame, pixel_x, pixel_y = 20, 82, 130 # pixel at frame 11 at position 100, 100
N = 5 # for the N x N region
half_N = N // 2

# make sure we stay within bounds
x_start = max(0, pixel_x - half_N)
x_end = min(image_shape[0], pixel_x + half_N + 1)
y_start = max(0, pixel_y - half_N)
y_end = min(image_shape[1], pixel_y + half_N + 1)

Vx_region = V_x_prewitt[selected_frame, x_start:x_end, y_start:y_end]
Vy_region = V_y_prewitt[selected_frame, x_start:x_end, y_start:y_end]
Vt_region = V_t[selected_frame, x_start:x_end, y_start:y_end]

# flatten the regions so we can create A and b
Vx_region_flattened = Vx_region.flatten()
Vy_region_flattened = Vy_region.flatten()
Vt_region_flattened = Vt_region.flatten()

A = np.vstack((Vx_region_flattened, Vy_region_flattened)).T
b = -Vt_region_flattened

u, v = np.linalg.lstsq(A, b, rcond=None)[0]

flow_vector, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)
u, v = flow_vector

print(f"Displacement vector (u, v) at pixel ({pixel_x}, {pixel_y}, {selected_frame}): ({u:.2f}, {v:.2f})")

plt.figure()
plt.imshow(image_sequence[selected_frame], cmap="gray")
plt.quiver(pixel_x, pixel_y, u, v, color="red", angles="xy", scale_units="xy", scale=1)
plt.title(f"Displacement vector (u, v) = ({u:.2f}, {v:.2f}) at pixel ({pixel_x}, {pixel_y}) for frame {selected_frame}")
plt.axis("off")
plt.show()

## Problem 3.2

In [ ]:
def calculate_optical_flow_single_frame(V_x, V_y, V_t, frame, N=5, stride=1):
    half_N = N // 2
    flow_vectors = np.zeros((frame.shape[0], frame.shape[1], 2))  # To store flow vectors (u, v)
    
    for x in range(half_N, frame.shape[0] - half_N, stride):
        for y in range(half_N, frame.shape[1] - half_N, stride):
            
            # Extract gradients in the N x N region around (x, y)
            x_start = max(0, x - half_N)
            x_end = min(frame.shape[0], x + half_N + 1)
            y_start = max(0, y - half_N)
            y_end = min(frame.shape[1], y + half_N + 1)
            
            Vx_region = V_x[x_start:x_end, y_start:y_end]
            Vy_region = V_y[x_start:x_end, y_start:y_end]
            Vt_region = V_t[x_start:x_end, y_start:y_end]

            Vx_region_flattened = Vx_region.flatten()
            Vy_region_flattened = Vy_region.flatten()
            Vt_region_flattened = Vt_region.flatten()
            
            A = np.vstack((Vx_region_flattened, Vy_region_flattened)).T
            b = -Vt_region_flattened
            
            flow_vector, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)
            u, v = flow_vector
            
            flow_vectors[x, y] = [u, v]
    
    return flow_vectors

selected_frame = 13
frame = image_sequence[selected_frame]
V_x = V_x_prewitt[selected_frame]
V_y = V_y_prewitt[selected_frame]
V_t = convolve(image_sequence, temporal_filter.reshape(2, 1, 1), mode="constant") # just resetting the V_t
V_t = V_t[selected_frame]

flow_vectors = calculate_optical_flow_single_frame(V_x, V_y, V_t, frame, N=5, stride=2)

# Plot the vector field
plt.figure(figsize=(10, 10))
plt.imshow(frame, cmap="gray")
X, Y = np.meshgrid(np.arange(flow_vectors.shape[1]), np.arange(flow_vectors.shape[0]))
U = flow_vectors[:, :, 0]
V = flow_vectors[:, :, 1]
plt.quiver(X, Y, U, V, color='r', scale=1, scale_units='xy')
plt.title(f"Optical Flow Vector Field for Frame {selected_frame}")
plt.axis("off")
plt.show()

In [ ]:
def calculate_optical_flow_all_frames(image_sequence, V_x_prewitt, V_y_prewitt, V_t, N=5, stride=1):
    num_frames = image_sequence.shape[0]
    flow_vectors_all_frames = []
    
    for t in range(num_frames):
        frame = image_sequence[t]
        V_x = V_x_prewitt[t]
        V_y = V_y_prewitt[t]
        V_t_frame = V_t[t]
        
        flow_vectors = calculate_optical_flow_single_frame(V_x, V_y, V_t_frame, frame, N, stride)
        flow_vectors_all_frames.append(flow_vectors)
    
    return flow_vectors_all_frames

In [ ]:
V_t = convolve(image_sequence, temporal_filter.reshape(2, 1, 1), mode="constant") # just resetting the V_t

flow_vectors_all_frames = calculate_optical_flow_all_frames(image_sequence, V_x_prewitt, V_y_prewitt, V_t, N=5, stride=2)

# Plot the vector field for each frame
for t in range(len(flow_vectors_all_frames)):
    frame = image_sequence[t]
    flow_vectors = flow_vectors_all_frames[t]
    
    plt.figure(figsize=(10, 10))
    plt.imshow(frame, cmap="gray")
    X, Y = np.meshgrid(np.arange(flow_vectors.shape[1]), np.arange(flow_vectors.shape[0]))
    U = flow_vectors[:, :, 0]
    V = flow_vectors[:, :, 1]
    plt.quiver(X, Y, U, V, color='r', scale=1, scale_units='xy')
    plt.title(f"Optical Flow Vector Field for Frame {t}")
    plt.axis("off")
    plt.show()

In [ ]:
import matplotlib.animation as animation

V_t = convolve(image_sequence, temporal_filter.reshape(2, 1, 1), mode="constant") # just resetting the V_t

fig, ax = plt.subplots(figsize=(10, 10))

def update_quiver(t):
    ax.clear()
    frame = image_sequence[t]
    flow_vectors = flow_vectors_all_frames[t]
    
    ax.imshow(frame, cmap="gray")
    X, Y = np.meshgrid(np.arange(flow_vectors.shape[1]), np.arange(flow_vectors.shape[0]))
    U = flow_vectors[:, :, 0]
    V = flow_vectors[:, :, 1]
    ax.quiver(X, Y, U, V, color='r', scale=1, scale_units='xy')
    ax.set_title(f"Optical Flow Vector Field for Frame {t}")
    ax.axis("off")

ani = animation.FuncAnimation(fig, update_quiver, frames=len(flow_vectors_all_frames), interval=200)

# Save the animation to a file on the desktop
output_path = os.path.expanduser("~/Desktop/optical_flow_animatio_2.mp4")
ani.save(output_path, writer='ffmpeg', fps=5)

plt.close(fig)

We can also do that with the Gaussian

In [ ]:
selected_frame = 10
frame = image_sequence[selected_frame]
V_x = V_x_gauss(1,2)
V_y = V_y_gauss(1,2)
V_t = convolve(image_sequence, temporal_filter.reshape(2, 1, 1), mode="constant")


flow_vectors_all_frames = calculate_optical_flow_all_frames(image_sequence, V_x, V_y, V_t, N=3, stride=3)

# Plot the vector field for each frame
for t in range(len(flow_vectors_all_frames)):
    frame = image_sequence[t]
    flow_vectors = flow_vectors_all_frames[t]
    
    plt.figure(figsize=(10, 10))
    plt.imshow(frame, cmap="gray")
    X, Y = np.meshgrid(np.arange(flow_vectors.shape[1]), np.arange(flow_vectors.shape[0]))
    U = flow_vectors[:, :, 0]
    V = flow_vectors[:, :, 1]
    plt.quiver(X, Y, U, V, color='r', scale=1, scale_units='xy')
    plt.title(f"Optical Flow Vector Field for Frame {t}")
    plt.axis("off")
    plt.show()

V_t = convolve(image_sequence, temporal_filter.reshape(2, 1, 1), mode="constant") # just resetting the V_t

fig, ax = plt.subplots(figsize=(10, 10))

def update_quiver(t):
    ax.clear()
    frame = image_sequence[t]
    flow_vectors = flow_vectors_all_frames[t]
    
    ax.imshow(frame, cmap="gray")
    X, Y = np.meshgrid(np.arange(flow_vectors.shape[1]), np.arange(flow_vectors.shape[0]))
    U = flow_vectors[:, :, 0]
    V = flow_vectors[:, :, 1]
    ax.quiver(X, Y, U, V, color='r', scale=1, scale_units='xy')
    ax.set_title(f"Optical Flow Vector Field for Frame {t}")
    ax.axis("off")

ani = animation.FuncAnimation(fig, update_quiver, frames=len(flow_vectors_all_frames), interval=200)

# Save the animation to a file on the desktop
output_path = os.path.expanduser("~/Desktop/gaussian-ballade.mp4")
ani.save(output_path, writer='ffmpeg', fps=5)

plt.close(fig)

## Part 4

## Problem 4.1

In [ ]:
image_files = sorted([f for f in os.listdir("should-work-seq")])

num_images = len(image_files)
image_shape = (256, 256)
own_image_sequence = np.zeros((num_images, *image_shape), dtype=np.float32)

for i, file in enumerate(image_files):
    img_path = os.path.join("should-work-seq", file)
    img = Image.open(img_path).convert("L") # this converts to grayscale
    img_array = np.array(img, dtype=np.float32) / 255.0 # this normalises to [0, 1]
    own_image_sequence[i] = img_array

In [ ]:
V_t = convolve(own_image_sequence, temporal_filter.reshape(2, 1, 1), mode="constant") # just resetting the V_t

flow_vectors_all_frames = calculate_optical_flow_all_frames(own_image_sequence, V_x_prewitt, V_y_prewitt, V_t, N=5, stride=5)

# Plot the vector field for each frame
for t in range(len(flow_vectors_all_frames)):
    frame = own_image_sequence[t]
    flow_vectors = flow_vectors_all_frames[t]
    
    plt.figure(figsize=(10, 10))
    plt.imshow(frame, cmap="gray")
    X, Y = np.meshgrid(np.arange(flow_vectors.shape[1]), np.arange(flow_vectors.shape[0]))
    U = flow_vectors[:, :, 0]
    V = flow_vectors[:, :, 1]
    plt.quiver(X, Y, U, V, color='r', scale=1, scale_units='xy')
    plt.title(f"Optical Flow Vector Field for Frame {t}")
    plt.axis("off")
    plt.show()

In [ ]:
V_t = convolve(own_image_sequence, temporal_filter.reshape(2, 1, 1), mode="constant") # just resetting the V_t

fig, ax = plt.subplots(figsize=(10, 10))

def update_quiver(t):
    ax.clear()
    frame = own_image_sequence[t]
    flow_vectors = flow_vectors_all_frames[t]
    
    ax.imshow(frame, cmap="gray")
    X, Y = np.meshgrid(np.arange(flow_vectors.shape[1]), np.arange(flow_vectors.shape[0]))
    U = flow_vectors[:, :, 0]
    V = flow_vectors[:, :, 1]
    ax.quiver(X, Y, U, V, color='r', scale=1, scale_units='xy')
    ax.set_title(f"Optical Flow Vector Field for Frame {t}")
    ax.axis("off")

ani = animation.FuncAnimation(fig, update_quiver, frames=len(flow_vectors_all_frames), interval=200)

# Save the animation to a file on the desktop
output_path = os.path.expanduser("~/Desktop/optical_flow_animation_own1.mp4")
ani.save(output_path, writer='ffmpeg', fps=5)

plt.close(fig)

Trying with Sobel instead of prewitt

In [ ]:
V_t = convolve(own_image_sequence, temporal_filter.reshape(2, 1, 1), mode="constant") # just resetting the V_t

flow_vectors_all_frames = calculate_optical_flow_all_frames(own_image_sequence, V_x_sobel, V_y_sobel, V_t, N=5, stride=5)

# Plot the vector field for each frame
for t in range(len(flow_vectors_all_frames)):
    frame = own_image_sequence[t]
    flow_vectors = flow_vectors_all_frames[t]
    
    plt.figure(figsize=(10, 10))
    plt.imshow(frame, cmap="gray")
    X, Y = np.meshgrid(np.arange(flow_vectors.shape[1]), np.arange(flow_vectors.shape[0]))
    U = flow_vectors[:, :, 0]
    V = flow_vectors[:, :, 1]
    plt.quiver(X, Y, U, V, color='r', scale=1, scale_units='xy')
    plt.title(f"Optical Flow Vector Field for Frame {t}")
    plt.axis("off")
    plt.show()

In [ ]:
V_t = convolve(own_image_sequence, temporal_filter.reshape(2, 1, 1), mode="constant") # just resetting the V_t

flow_vectors_all_frames = calculate_optical_flow_all_frames(own_image_sequence, V_x_sobel, V_y_sobel, V_t, N=5, stride=5)

# Plot the vector field for each frame
for t in range(len(flow_vectors_all_frames)):
    frame = own_image_sequence[t]
    flow_vectors = flow_vectors_all_frames[t]
    
    plt.figure(figsize=(10, 10))
    plt.imshow(frame, cmap="gray")
    X, Y = np.meshgrid(np.arange(flow_vectors.shape[1]), np.arange(flow_vectors.shape[0]))
    U = flow_vectors[:, :, 0]
    V = flow_vectors[:, :, 1]
    plt.quiver(X, Y, U, V, color='r', scale=1, scale_units='xy')
    plt.title(f"Optical Flow Vector Field for Frame {t}")
    plt.axis("off")
    plt.show()

In [ ]:
image_files = sorted([f for f in os.listdir("should-break-seq")])

num_images = len(image_files)
image_shape = (256, 256)
breaking_img_sequence = np.zeros((num_images, *image_shape), dtype=np.float32)

for i, file in enumerate(image_files):
    if file.startswith("."):
        continue
    img_path = os.path.join("should-break-seq", file)
    img = Image.open(img_path).convert("L") # this converts to grayscale
    img_array = np.array(img, dtype=np.float32) / 255.0 # this normalises to [0, 1]
    breaking_img_sequence[i] = img_array

In [ ]:
print(breaking_img_sequence[0])

In [ ]:
V_x_prewitt = convolve(breaking_img_sequence, prewitt_x[np.newaxis, :, :], mode="constant")
V_y_prewitt = convolve(breaking_img_sequence, prewitt_y[np.newaxis, :, :], mode="constant")

In [ ]:
V_t = convolve(breaking_img_sequence, temporal_filter.reshape(2, 1, 1), mode="constant") # just resetting the V_t

flow_vectors_all_frames = calculate_optical_flow_all_frames(breaking_img_sequence, V_x_prewitt, V_y_prewitt, V_t, N=5, stride=2)

# Plot the vector field for each frame
for t in range(len(flow_vectors_all_frames)):
    frame = breaking_img_sequence[t]
    flow_vectors = flow_vectors_all_frames[t]
    
    plt.figure(figsize=(10, 10))
    plt.imshow(frame, cmap="gray")
    X, Y = np.meshgrid(np.arange(flow_vectors.shape[1]), np.arange(flow_vectors.shape[0]))
    U = flow_vectors[:, :, 0]
    V = flow_vectors[:, :, 1]
    plt.quiver(X, Y, U, V, color='r', scale=1, scale_units='xy')
    plt.title(f"Optical Flow Vector Field for Frame {t}")
    plt.axis("off")
    plt.show()

In [ ]:
V_t = convolve(own_image_sequence, temporal_filter.reshape(2, 1, 1), mode="constant") # just resetting the V_t

fig, ax = plt.subplots(figsize=(10, 10))

def update_quiver(t):
    ax.clear()
    frame = own_image_sequence[t]
    flow_vectors = flow_vectors_all_frames[t]
    
    ax.imshow(frame, cmap="gray")
    X, Y = np.meshgrid(np.arange(flow_vectors.shape[1]), np.arange(flow_vectors.shape[0]))
    U = flow_vectors[:, :, 0]
    V = flow_vectors[:, :, 1]
    ax.quiver(X, Y, U, V, color='r', scale=1, scale_units='xy')
    ax.set_title(f"Optical Flow Vector Field for Frame {t}")
    ax.axis("off")

ani = animation.FuncAnimation(fig, update_quiver, frames=len(flow_vectors_all_frames), interval=200)

# Save the animation to a file on the desktop
output_path = os.path.expanduser("~/Desktop/breaking-video.mp4")
ani.save(output_path, writer='ffmpeg', fps=5)

plt.close(fig)